In [ ]:
import matplotlib.pyplot as plt

from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read_events
from obspy import read_inventory
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import pickle


In [ ]:
catINGV=read_events('flegrei_2014_2023_INGV/catalogue_flegrei.xml')

In [ ]:
catOV=read_events('flegrei_2014_2023_GOSSIP/catalogue_flegrei.xml')

In [ ]:
id_catalogues=[]

for ind,ev in enumerate(catINGV):
    
    origin_time = ev.origins[0].time
    time_min= 'time > ' + str(origin_time - 5)
    time_max= 'time < ' + str(origin_time + 5)
    cat= catOV.filter(time_min , time_max)

    id_catalogues.append([])
    id_catalogues[ind].append( int( str(ev.resource_id).split('=')[1] ) )

    for match in cat:
        id_catalogues[ind].append( int( str(match.resource_id).split('/')[1] ) )

id_catalogues

## save

In [ ]:
with open("event_comparison_INGV_GOSSIP", "wb") as fp:   #Pickling
    pickle.dump(id_catalogues, fp)

## load

In [ ]:
with open("event_comparison_INGV_GOSSIP", "rb") as fp:   # Unpickling
    catalogues = pickle.load(fp)

In [ ]:
ev_not_found=[]
ev_multiple_det=[]

for ev in catalogues:
    if len(ev)   >= 3:
        ev_multiple_det.append(ev)
    elif len(ev) == 1:
        ev_not_found.append(ev)
    elif len(ev) != 2:
        print('!event not categorized!')
        

In [ ]:
#[ev for ev in catOV if ev.resource_id.id.split('/')[1]== str(catalogues[0][1]) ]

# Stations

In [ ]:
client=Client('INGV')

stime=UTCDateTime('2014-01-01T00:00:00')
etime=UTCDateTime('2023-11-20T00:00:00')

In [ ]:
lat=40.8478
long=14.0918
rad_events=0.5
inv=client.get_stations(starttime=stime,endtime=etime,
                        latitude=lat,longitude=long,maxradius=rad_events,
                        level='response')                                   #instrumental response
inv

In [ ]:
inv.plot(projection='local',resolution='h');

In [ ]:
inv.write('flegrei_2014_2023_INGV/inventory_flegrei.xml',format='STATIONXML')                        #save

In [ ]:
inv_f=read_inventory('flegrei_2014_2023_INGV/inventory_flegrei.xml')                                 #read

In [ ]:
#inv_f.plot_response(min_freq=1E-4,station='SORR',channel='[EH]H?');

# Waveform 

In [25]:
catOV.filter("magnitude >= 1.5","magnitude < 2.0")

149 Event(s) in Catalog:
2016-06-04T02:38:52.990000Z | +40.822,  +14.134 | 1.5  D 
2016-06-04T02:45:58.830000Z | +40.824,  +14.131 | 1.6  D 
...
2020-12-20T03:20:26.690000Z | +40.824,  +14.137 | 1.5  D 
2020-12-23T01:59:42.790000Z | +40.828,  +14.140 | 1.6  D 
To see all events call 'print(CatalogObject.__str__(print_all=True))'

In [28]:
id='2508'
ev_match=Catalog()
for ev in catOV:
    if id == ev.resource_id.id.split('/')[1]:
        ev_match += ev
ev_match.events[0].origins


[Origin
	 resource_id: ResourceIdentifier(id="smi:local/299f3b3d-a17a-4ff7-a133-86ec348bd497")
	        time: UTCDateTime(2019, 12, 6, 0, 17, 23, 780000)
	   longitude: 14.14933
	    latitude: 40.83017
	       depth: 2330.0]

In [23]:
catOV_mag = catOV.filter("magnitude >= 2.0","magnitude < 2.5")
for ev in catOV_mag:
    origin_time=ev.origins[0].time
    evID=ev.resource_id.id.split('/')[1]
    print('event id: ',evID)
    print('origin time event:',origin_time)
    print('extimated magnitude:',ev.magnitudes[0].mag)


    event_start=origin_time -10
    event_end=origin_time +50

    wave=Stream()
    for network in inv_f:
        for  station in network.stations:
            try:
                wave += client.get_waveforms(starttime=event_start,endtime=event_end,
                                    network=network.code,station=station.code,location='*', channel='HH?',
                                    attach_response=True,)
            except:
                #print(station.code , 'station not found')
                continue

    print('traces found:',len(wave.traces))

    wave.merge(fill_value=0)
    # trim over the [t1, t2] interval
    wave.trim(starttime=event_start, endtime=event_end, pad=True, fill_value=0)

    '''
    # remove trend
    wave.detrend("demean")
    
    #remove instrumental response
    pre_filt = [0.01, 0.1, 25,30]
    
    #remove instrumental response
    wave.remove_response(inventory=inv_f, output='DISP', pre_filt=pre_filt) #output=VEL ??
    '''

    wave.write('waveform/mag2/wave_'+ evID +'.mseed',format='MSEED')
    print('saved!')

event id:  4201
origin time event: 2016-07-05T01:22:39.730000Z
extimated magnitude: 2.1
traces found: 48
saved!
event id:  4242
origin time event: 2015-10-07T07:37:40.700000Z
extimated magnitude: 2.2


/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


traces found: 9
saved!
event id:  4232
origin time event: 2015-10-07T07:54:13.490000Z
extimated magnitude: 2.0


/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


traces found: 9
saved!
event id:  4218
origin time event: 2015-10-07T08:13:51.190000Z
extimated magnitude: 2.3


...

/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


traces found: 54
saved!
event id:  4354
origin time event: 2020-12-20T03:13:48.650000Z
extimated magnitude: 2.4


/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')


traces found: 54
saved!


/Users/user/Library/Python/3.9/lib/python/site-packages/obspy/io/mseed/core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')
